In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image


In [3]:
# Update with the correct paths for your dataset
dataset_folder = '/content/drive/MyDrive/Dataset'
train_csv_path = os.path.join(dataset_folder, '/content/drive/MyDrive/Dataset/primary_train.csv')
val_csv_path = os.path.join(dataset_folder, '/content/drive/MyDrive/Dataset/val(unseen).csv')
test_csv_path = os.path.join(dataset_folder, '/content/drive/MyDrive/Dataset/test(unseen).csv')
image_folder = os.path.join(dataset_folder, 'all_images')

In [4]:
# Define a custom dataset class for the dataset with the new structure
class CustomDatasetFromCSV(Dataset):
    def __init__(self, csv_path, image_folder, transform=None):
        self.data = pd.read_csv(csv_path)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = int(self.data.iloc[idx, 1])  # Assuming the label column is at index 1
        return image, label

In [5]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# Update model initialization for ResNet-18 with Dropout, L2 Regularization, and BatchNorm
class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes, dropout_prob=0.5, l2_reg=0.001):
        super(ResNet18Classifier, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        in_features = self.resnet18.fc.in_features

        # Replace the fully connected layer with dropout and L2 regularization
        self.resnet18.fc = nn.Sequential(
            nn.Linear(in_features, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.BatchNorm1d(num_classes),  # Batch normalization after dropout
        )

        # Add L2 regularization to all Conv2d layers
        for module in self.resnet18.modules():
            if isinstance(module, nn.Conv2d):
                module.weight_regularizer = torch.nn.Parameter(
                    l2_reg * torch.ones_like(module.weight)
                )

    def forward(self, x):
        return self.resnet18(x)

# Instantiate the ResNet18-based model with ensemble learning components
model = ResNet18Classifier(num_classes=9, dropout_prob=0.5, l2_reg=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 247MB/s]


In [7]:
# Create datasets using the new structure
train_dataset = CustomDatasetFromCSV(train_csv_path, image_folder, transform=transform)
val_dataset = CustomDatasetFromCSV(val_csv_path, image_folder, transform=transform)
test_dataset = CustomDatasetFromCSV(test_csv_path, image_folder, transform=transform)

In [8]:
batch_size=32
# Split the datasets and create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# Step 3: Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Update the optimizer to use ResNet-18 parameters
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0)  # L2 regularization as weight_decay

# Add learning rate decay
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet18Classifier(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [10]:
# Initialize variables to keep track of the best model
best_validation_loss = float('inf')
best_model_path = '/content/drive/MyDrive/Dataset/ResNet18_model_3.pth'

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    predictions = []
    true_labels = []

    for images, labels in train_loader:  # Assuming you have defined train_loader
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Zero the gradient buffers
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Loss computation
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        total_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Compute training accuracy and loss
    training_accuracy = accuracy_score(true_labels, predictions)
    training_loss = total_loss / len(train_loader)  # Compute the average loss

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_total_loss = 0.0
    val_predictions = []
    val_true_labels = []

    for images, labels in val_loader:  # Assuming you have defined val_loader
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            val_total_loss += val_loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_predictions.extend(predicted.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    # Compute validation accuracy and loss
    validation_accuracy = accuracy_score(val_true_labels, val_predictions)
    validation_loss = val_total_loss / len(val_loader)  # Compute the average loss

    # Save the model with the best validation loss
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        torch.save(model.state_dict(), best_model_path)

    scheduler.step()

    # Print the metrics for this epoch
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"Training Loss: {training_loss:.4f}, Training Accuracy: {training_accuracy:.4f}")
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")
    print(f"Learning Rate: {scheduler.get_last_lr()}")

# Save the trained model
torch.save(model.state_dict(), 'resnet18_model_3.pth')

# Save the best model based on validation loss
if os.path.exists(best_model_path):
    os.remove(best_model_path)  # Remove previous best model
torch.save(model.state_dict(), best_model_path)
print(f"Best model saved at: {best_model_path}")


Epoch 1/100:
Training Loss: 1.3380, Training Accuracy: 0.5414
Validation Loss: 1.6891, Validation Accuracy: 0.4765
Learning Rate: [0.001]
Epoch 2/100:
Training Loss: 1.1888, Training Accuracy: 0.6043
Validation Loss: 1.4478, Validation Accuracy: 0.5570
Learning Rate: [0.001]
Epoch 3/100:
Training Loss: 1.0915, Training Accuracy: 0.6264
Validation Loss: 1.3684, Validation Accuracy: 0.6309
Learning Rate: [0.001]
Epoch 4/100:
Training Loss: 1.0963, Training Accuracy: 0.6196
Validation Loss: 1.6498, Validation Accuracy: 0.4966
Learning Rate: [0.001]
Epoch 5/100:
Training Loss: 1.0930, Training Accuracy: 0.6173
Validation Loss: 1.4890, Validation Accuracy: 0.4631
Learning Rate: [0.001]
Epoch 6/100:
Training Loss: 1.0899, Training Accuracy: 0.6128
Validation Loss: 1.6540, Validation Accuracy: 0.4497
Learning Rate: [0.001]
Epoch 7/100:
Training Loss: 1.0694, Training Accuracy: 0.6168
Validation Loss: 1.5461, Validation Accuracy: 0.5101
Learning Rate: [0.001]
Epoch 8/100:
Training Loss: 1.0753

In [11]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Load the test data
test_csv_file = '/content/drive/MyDrive/Dataset/test(unseen).csv'
test_data = pd.read_csv(test_csv_file)

# Custom dataset class for test data
class TestDataset(Dataset):
    def __init__(self, data, image_dir, transform=None):
        self.data = data
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

# Define transforms for test data (similar to training)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create a dataset and data loader for test data
test_image_dir = '/content/drive/MyDrive/Dataset/all_images'
test_dataset = TestDataset(test_data, test_image_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=32)

# Load the trained model (CustomResNet18)
model = ResNet18Classifier(num_classes=9)  # Use the custom model
model.load_state_dict(torch.load('/content/drive/MyDrive/Dataset/ResNet18_model_3.pth'))

model.eval()  # Set the model to evaluation mode

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Testing loop
predictions = []

with torch.no_grad():
    for images in test_dataloader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

# Print the predictions
print("Predictions:", predictions)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predictions: [5, 2, 8, 0, 2, 2, 3, 1, 0, 0, 7, 2, 0, 7, 0, 2, 3, 3, 0, 8, 3, 0, 7, 3, 4, 8, 1, 7, 3, 7, 7, 3, 2, 3, 8, 0, 5, 3, 3, 8, 4]


In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming ground truth labels for the test data are in test_data['label']
# Replace 'test_data['label']' with the actual ground truth labels from your test data

# Calculate accuracy
ground_truth_labels = test_data['label'].tolist()  # Ground truth labels
accuracy = accuracy_score(ground_truth_labels, predictions)

# Calculate class-wise accuracy using a confusion matrix
confusion = confusion_matrix(ground_truth_labels, predictions)
class_accuracy = confusion.diagonal() / confusion.sum(axis=1)

# Print the overall accuracy
print("Overall Accuracy:", accuracy * 100, "%")

# Print class-wise accuracy
for i, acc in enumerate(class_accuracy):
    print(f"Class {i}: {acc * 100:.2f}%")

# Print a detailed classification report
report = classification_report(ground_truth_labels, predictions, target_names=["Class 0", "Class 1", "Class 2", "Class 3", "Class 4", "Class 5", "Class 6", "Class 7"])
print(report)


Overall Accuracy: 53.65853658536586 %
Class 0: 100.00%
Class 1: 28.57%
Class 2: 100.00%
Class 3: 62.50%
Class 4: 50.00%
Class 5: 28.57%
Class 6: 54.55%
Class 7: nan%
              precision    recall  f1-score   support

     Class 0       0.50      1.00      0.67         4
     Class 1       1.00      0.29      0.44         7
     Class 2       0.33      1.00      0.50         2
     Class 3       0.50      0.62      0.56         8
     Class 4       0.50      0.50      0.50         2
     Class 5       1.00      0.29      0.44         7
     Class 6       1.00      0.55      0.71        11
     Class 7       0.00      0.00      0.00         0

    accuracy                           0.54        41
   macro avg       0.60      0.53      0.48        41
weighted avg       0.80      0.54      0.56        41



<ipython-input-13-2f1cc2e62bf1>:12: RuntimeWarning: invalid value encountered in divide
  class_accuracy = confusion.diagonal() / confusion.sum(axis=1)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this b